**Comparison of Mean Squared Error (MSE) Across Different Model Configurations**


### Introduction
This project compares the performance of different configurations of a neural network regression model for predicting concrete strength. The performance is measured using the mean squared error (MSE) across multiple trials with varying model parameters such as data normalization, number of epochs, and number of hidden layers. 


### Experiment Setup
The model will be evaluated under the following conditions:
- **Baseline Model:** 1 hidden layer, 50 epochs, no data normalization.
- **Normalized Data:** 1 hidden layer, 50 epochs, with data normalization.
- **Increased Epochs:** 1 hidden layer, 100 epochs, with data normalization.
- **Increased Hidden Layers:** 3 hidden layers, 50 epochs, with data normalization.

Each experiment will be conducted across 50 trials with different random seeds for train-test splitting. The average MSE and standard deviation of MSE will be recorded.


In [1]:
!pip install scikit-learn
!pip install numpy==2.0.2
!pip install pandas==2.2.2
!pip install tensorflow_cpu==2.18.0
!pip install matplotlib==3.9.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 144.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 191.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 131.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 145.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.3
    Uninstalling numpy-2.2.3:
      Successfully uninstalled numpy-2.2.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 169.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.3/230.3 MB 44.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 48.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os

# Set environment variables
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppresses INFO and WARNING messages
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disables oneDNN custom operations

# 1. Import Libraries and Packages

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Input

# 2. Data Preparation

In [5]:
# Load dataset
data = pd.read_csv('concrete_data.csv')
data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [7]:
data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


# A. Build a Baseline Model

In [18]:
# Prepare predictors and target
predictors = data.drop(columns=['Strength'])
target = data['Strength']


In [19]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [20]:
n_cols = predictors.shape[1] # number of predictors
mean_squared_errors = []

for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=1)
    model = regression_model()
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=2)
    mse = model.evaluate(x_test, y_test, verbose=0)
    mean_squared_errors.append(mse)

# Compute the mean and standard deviation of the mean squared errors
import numpy as np
mean_mse = np.mean(mean_squared_errors)
std_mse = np.std(mean_squared_errors)

print("Mean of Mean Squared Errors: {}".format(mean_mse))
print("Standard Deviation of Mean Squared Errors: {}".format(std_mse))


Epoch 1/50


/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 60ms/step - loss: 15967.5039 - val_loss: 4167.1367
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 3591.0637 - val_loss: 3495.9250
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 3143.9548 - val_loss: 2915.4390
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 2761.2473 - val_loss: 2554.6409
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 2461.3076 - val_loss: 2284.1147
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 2241.5066 - val_loss: 2071.1436
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 2068.0447 - val_loss: 1901.3309
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1919.7618 - val_loss: 1750.8495
Epoch 9/50
23/23 - 0s - 5ms/step - loss: 1792.7406 - val_loss: 1625.2870
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1668.6807 - val_loss: 1484.2738
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1555.2217 - val_loss: 1376.4094
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 1455.8325 - val_loss: 1279.0017
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1361.4895 - val_loss: 1181.3682
Epoch 14/50
23/23 - 0s - 7ms/step - loss: 1254.7714 - va

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 36ms/step - loss: 326236.9688 - val_loss: 209994.2656
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 148773.1875 - val_loss: 81585.1641
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 53485.7031 - val_loss: 24172.5176
Epoch 4/50
23/23 - 0s - 6ms/step - loss: 15520.6602 - val_loss: 6727.4648
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 5184.8501 - val_loss: 3736.6453
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 3462.9614 - val_loss: 3574.6978
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 3285.8052 - val_loss: 3543.0813
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 3222.5552 - val_loss: 3464.9260
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 3157.9404 - val_loss: 3376.9199
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 3088.5164 - val_loss: 3290.2998
Epoch 11/50
23/23 - 0s - 5ms/step - loss: 3024.1172 - val_loss: 3180.0701
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 2953.9983 - val_loss: 3130.1379
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 2880.2761 - val_loss: 3024.5896
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 2809.

...

23/23 - 2s - 92ms/step - loss: 7587.4424 - val_loss: 2722.2769
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1761.4851 - val_loss: 1450.5847
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1345.3307 - val_loss: 1391.5707
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1263.1703 - val_loss: 1286.5514
Epoch 5/50
23/23 - 0s - 7ms/step - loss: 1196.7992 - val_loss: 1205.2389
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1126.3683 - val_loss: 1126.8210
Epoch 7/50
23/23 - 0s - 8ms/step - loss: 1052.4581 - val_loss: 1049.6479
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 990.2721 - val_loss: 977.8917
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 927.9739 - val_loss: 910.1733
Epoch 10/50
23/23 - 0s - 7ms/step - loss: 870.1194 - val_loss: 848.2119
Epoch 11/50
23/23 - 0s - 7ms/step - loss: 815.0196 - val_loss: 788.4497
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 765.2987 - val_loss: 735.2993
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 720.5789 - val_loss: 684.6629
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 677.9737 - val_loss: 643.46

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 40ms/step - loss: 248254.0938 - val_loss: 166108.2031
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 109383.4375 - val_loss: 55283.9688
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 27359.2930 - val_loss: 8299.4189
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 4313.9302 - val_loss: 3431.2869
Epoch 5/50
23/23 - 0s - 7ms/step - loss: 3180.5017 - val_loss: 3175.7925
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 2869.4680 - val_loss: 2991.6348
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 2659.9719 - val_loss: 2809.2839
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 2490.4836 - val_loss: 2641.1951
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 2315.8760 - val_loss: 2486.2458
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 2164.4377 - val_loss: 2341.0271
Epoch 11/50
23/23 - 0s - 5ms/step - loss: 2031.5627 - val_loss: 2210.8613
Epoch 12/50
23/23 - 0s - 7ms/step - loss: 1901.5276 - val_loss: 2103.3625
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1792.4752 - val_loss: 1988.7214
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 1688.66

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 38ms/step - loss: 825.3653 - val_loss: 419.8486
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 428.6337 - val_loss: 270.4287
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 291.8423 - val_loss: 231.0036
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 246.7852 - val_loss: 216.7728
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 228.8117 - val_loss: 196.9760
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 216.4716 - val_loss: 186.1353
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 205.1070 - val_loss: 177.0077
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 194.9509 - val_loss: 170.2594
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 186.9593 - val_loss: 165.6634
Epoch 10/50
23/23 - 0s - 5ms/step - loss: 178.2591 - val_loss: 160.1497
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 171.5987 - val_loss: 157.6288
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 168.2810 - val_loss: 152.9796
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 160.3799 - val_loss: 146.9158
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 153.5118 - val_loss: 152.1230
Epoch 15/50

In [21]:
# Compute the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mean_squared_errors)
std_mse = np.std(mean_squared_errors)

print("Mean of Mean Squared Errors: {}".format(mean_mse))
print("Standard Deviation of Mean Squared Errors: {}".format(std_mse))

Mean of Mean Squared Errors: 366.75229232788087
Standard Deviation of Mean Squared Errors: 555.3843319777931


# B. Normalize the Data

In [22]:
predictors = data.drop('Strength',axis=1)
target = data['Strength']

predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [23]:
n_cols = predictors_norm.shape[1] # number of predictors
mean_squared_errors = []

for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size = 0.3, random_state = 1)
    model = regression_model()
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=2)
    mse = model.evaluate(x_test, y_test, verbose=0)
    mean_squared_errors.append(mse)

Epoch 1/50


/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 39ms/step - loss: 1554.1431 - val_loss: 1587.2181
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1537.3184 - val_loss: 1570.4467
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1520.2722 - val_loss: 1553.6422
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1503.4766 - val_loss: 1536.4381
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1486.1556 - val_loss: 1519.2207
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1468.5409 - val_loss: 1501.2684
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1450.5054 - val_loss: 1482.6577
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1431.6200 - val_loss: 1463.5972
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1412.4250 - val_loss: 1443.8440
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1392.7439 - val_loss: 1423.1854
Epoch 11/50
23/23 - 0s - 5ms/step - loss: 1371.8411 - val_loss: 1402.5034
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 1350.7834 - val_loss: 1380.4684
Epoch 13/50
23/23 - 0s - 5ms/step - loss: 1328.7473 - val_loss: 1357.5829
Epoch 14/50
23/23 - 0s - 11ms/step - loss: 1305.7134 - va

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 36ms/step - loss: 1553.0345 - val_loss: 1585.0095
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1537.8254 - val_loss: 1568.7611
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1522.4823 - val_loss: 1552.5410
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1506.9783 - val_loss: 1536.1949
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1491.2747 - val_loss: 1519.1312
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1475.0990 - val_loss: 1501.6489
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1458.1583 - val_loss: 1483.4814
Epoch 8/50
23/23 - 0s - 5ms/step - loss: 1440.4896 - val_loss: 1464.6073
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1421.9200 - val_loss: 1444.3219
Epoch 10/50
23/23 - 0s - 5ms/step - loss: 1402.2552 - val_loss: 1423.2255
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1381.4559 - val_loss: 1400.6356
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 1359.5815 - val_loss: 1377.3966
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1336.7021 - val_loss: 1353.1121
Epoch 14/50
23/23 - 0s - 7ms/step - loss: 1312.8131 - val

...

23/23 - 1s - 35ms/step - loss: 1503.6307 - val_loss: 1527.4573
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1488.4675 - val_loss: 1511.6625
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1473.1836 - val_loss: 1494.9012
Epoch 4/50
23/23 - 0s - 6ms/step - loss: 1457.1340 - val_loss: 1477.4861
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1440.1216 - val_loss: 1459.1105
Epoch 6/50
23/23 - 0s - 5ms/step - loss: 1421.8560 - val_loss: 1439.4561
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1402.5005 - val_loss: 1417.8291
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1381.4982 - val_loss: 1394.9761
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1358.9603 - val_loss: 1370.5765
Epoch 10/50
23/23 - 0s - 5ms/step - loss: 1335.0057 - val_loss: 1344.2650
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1309.4323 - val_loss: 1316.9117
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 1282.1152 - val_loss: 1287.1489
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1253.5865 - val_loss: 1255.7465
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 1223.0540 - val

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 34ms/step - loss: 1556.4304 - val_loss: 1600.7542
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1541.3724 - val_loss: 1585.0608
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1526.9613 - val_loss: 1570.1010
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1512.7594 - val_loss: 1555.4543
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1498.6246 - val_loss: 1541.0675
Epoch 6/50
23/23 - 0s - 5ms/step - loss: 1484.4933 - val_loss: 1526.5984
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1470.1934 - val_loss: 1511.7656
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1455.4513 - val_loss: 1497.0250
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1440.5327 - val_loss: 1481.4003
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1424.8875 - val_loss: 1465.5669
Epoch 11/50
23/23 - 0s - 5ms/step - loss: 1408.9502 - val_loss: 1448.7467
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 1391.7672 - val_loss: 1431.4435
Epoch 13/50
23/23 - 0s - 5ms/step - loss: 1373.8684 - val_loss: 1413.0735
Epoch 14/50
23/23 - 0s - 5ms/step - loss: 1354.8566 - val

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 34ms/step - loss: 1526.9559 - val_loss: 1559.4183
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1510.9426 - val_loss: 1544.3181
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1494.8231 - val_loss: 1528.9939
Epoch 4/50
23/23 - 0s - 6ms/step - loss: 1478.6028 - val_loss: 1513.0233
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1461.7372 - val_loss: 1496.3290
Epoch 6/50
23/23 - 0s - 5ms/step - loss: 1443.7753 - val_loss: 1479.1824
Epoch 7/50
23/23 - 0s - 5ms/step - loss: 1425.0647 - val_loss: 1460.6705
Epoch 8/50
23/23 - 0s - 5ms/step - loss: 1405.0953 - val_loss: 1440.9850
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1383.8180 - val_loss: 1420.2339
Epoch 10/50
23/23 - 0s - 5ms/step - loss: 1361.6310 - val_loss: 1397.9110
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1337.9224 - val_loss: 1374.7650
Epoch 12/50
23/23 - 0s - 7ms/step - loss: 1312.9640 - val_loss: 1350.3613
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1286.5873 - val_loss: 1324.8955
Epoch 14/50
23/23 - 0s - 5ms/step - loss: 1259.2549 - val

In [24]:
# Compute the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mean_squared_errors)
std_mse = np.std(mean_squared_errors)

print("Mean of Mean Squared Errors: {}".format(mean_mse))
print("Standard Deviation of Mean Squared Errors: {}".format(std_mse))

Mean of Mean Squared Errors: 356.37737548828125
Standard Deviation of Mean Squared Errors: 66.62830344581339


# C. Increase the number of Epochs

- Use epochs = 100

In [25]:
predictors = data.drop('Strength',axis=1)
target = data['Strength']

predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [26]:
n_cols = predictors_norm.shape[1] # number of predictors
mean_squared_errors = []

for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size = 0.3, random_state = 1)
    model = regression_model()
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, verbose=2)
    mse = model.evaluate(x_test, y_test, verbose=0)
    mean_squared_errors.append(mse)

Epoch 1/100


/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 35ms/step - loss: 1529.4896 - val_loss: 1548.9392
Epoch 2/100
23/23 - 0s - 6ms/step - loss: 1515.4019 - val_loss: 1534.0646
Epoch 3/100
23/23 - 0s - 6ms/step - loss: 1500.7917 - val_loss: 1518.7991
Epoch 4/100
23/23 - 0s - 6ms/step - loss: 1485.7365 - val_loss: 1502.8848
Epoch 5/100
23/23 - 0s - 6ms/step - loss: 1469.8853 - val_loss: 1486.0392
Epoch 6/100
23/23 - 0s - 5ms/step - loss: 1453.2753 - val_loss: 1468.3586
Epoch 7/100
23/23 - 0s - 5ms/step - loss: 1435.8390 - val_loss: 1449.7745
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 1417.4890 - val_loss: 1430.1166
Epoch 9/100
23/23 - 0s - 6ms/step - loss: 1398.0770 - val_loss: 1409.4078
Epoch 10/100
23/23 - 0s - 5ms/step - loss: 1377.5356 - val_loss: 1387.4708
Epoch 11/100
23/23 - 0s - 5ms/step - loss: 1355.7217 - val_loss: 1364.4327
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 1332.8749 - val_loss: 1340.3362
Epoch 13/100
23/23 - 0s - 5ms/step - loss: 1309.0381 - val_loss: 1314.6327
Epoch 14/100
23/23 - 0s - 6ms/step - loss: 12

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 44ms/step - loss: 1621.7603 - val_loss: 1652.9272
Epoch 2/100
23/23 - 0s - 6ms/step - loss: 1606.2250 - val_loss: 1637.3660
Epoch 3/100
23/23 - 0s - 7ms/step - loss: 1591.5294 - val_loss: 1622.4642
Epoch 4/100
23/23 - 0s - 6ms/step - loss: 1577.3483 - val_loss: 1608.1932
Epoch 5/100
23/23 - 0s - 6ms/step - loss: 1563.7122 - val_loss: 1594.1827
Epoch 6/100
23/23 - 0s - 6ms/step - loss: 1550.4348 - val_loss: 1580.2902
Epoch 7/100
23/23 - 0s - 6ms/step - loss: 1537.1387 - val_loss: 1566.7231
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 1524.1456 - val_loss: 1552.9440
Epoch 9/100
23/23 - 0s - 6ms/step - loss: 1511.0236 - val_loss: 1539.1825
Epoch 10/100
23/23 - 0s - 5ms/step - loss: 1497.9794 - val_loss: 1525.2391
Epoch 11/100
23/23 - 0s - 6ms/step - loss: 1484.5470 - val_loss: 1511.3584
Epoch 12/100
23/23 - 0s - 5ms/step - loss: 1471.0702 - val_loss: 1497.0074
Epoch 13/100
23/23 - 0s - 6ms/step - loss: 1457.1577 - val_loss: 1482.0598
Epoch 14/100
23/23 - 0s - 6ms/step - loss: 14

...

Epoch 1/100
23/23 - 1s - 44ms/step - loss: 1534.7744 - val_loss: 1566.5157
Epoch 2/100
23/23 - 0s - 7ms/step - loss: 1519.9395 - val_loss: 1551.5551
Epoch 3/100
23/23 - 0s - 7ms/step - loss: 1505.1189 - val_loss: 1536.2247
Epoch 4/100
23/23 - 0s - 7ms/step - loss: 1489.9537 - val_loss: 1520.4957
Epoch 5/100
23/23 - 0s - 9ms/step - loss: 1474.3698 - val_loss: 1504.2150
Epoch 6/100
23/23 - 0s - 6ms/step - loss: 1458.1179 - val_loss: 1487.2324
Epoch 7/100
23/23 - 0s - 6ms/step - loss: 1441.0746 - val_loss: 1469.5126
Epoch 8/100
23/23 - 0s - 7ms/step - loss: 1423.2002 - val_loss: 1450.5293
Epoch 9/100
23/23 - 0s - 6ms/step - loss: 1404.2281 - val_loss: 1430.4149
Epoch 10/100
23/23 - 0s - 6ms/step - loss: 1383.9712 - val_loss: 1409.0630
Epoch 11/100
23/23 - 0s - 6ms/step - loss: 1362.5641 - val_loss: 1385.8879
Epoch 12/100
23/23 - 0s - 6ms/step - loss: 1339.3422 - val_loss: 1361.5620
Epoch 13/100
23/23 - 0s - 6ms/step - loss: 1314.6278 - val_loss: 1335.4170
Epoch 14/100
23/23 - 0s - 6ms/ste

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 40ms/step - loss: 1567.2448 - val_loss: 1600.1442
Epoch 2/100
23/23 - 0s - 7ms/step - loss: 1551.1617 - val_loss: 1584.3682
Epoch 3/100
23/23 - 0s - 7ms/step - loss: 1535.7867 - val_loss: 1568.9351
Epoch 4/100
23/23 - 0s - 7ms/step - loss: 1520.7002 - val_loss: 1553.9626
Epoch 5/100
23/23 - 0s - 6ms/step - loss: 1505.7883 - val_loss: 1539.0182
Epoch 6/100
23/23 - 0s - 6ms/step - loss: 1490.5908 - val_loss: 1523.9604
Epoch 7/100
23/23 - 0s - 6ms/step - loss: 1475.3380 - val_loss: 1508.0597
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 1459.2590 - val_loss: 1491.9972
Epoch 9/100
23/23 - 0s - 6ms/step - loss: 1442.8484 - val_loss: 1475.0710
Epoch 10/100
23/23 - 0s - 6ms/step - loss: 1425.7113 - val_loss: 1457.7180
Epoch 11/100
23/23 - 0s - 6ms/step - loss: 1408.0972 - val_loss: 1439.7113
Epoch 12/100
23/23 - 0s - 6ms/step - loss: 1389.7223 - val_loss: 1421.3580
Epoch 13/100
23/23 - 0s - 6ms/step - loss: 1370.7634 - val_loss: 1401.8224
Epoch 14/100
23/23 - 0s - 5ms/step - loss: 13

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 40ms/step - loss: 1553.5820 - val_loss: 1579.2090
Epoch 2/100
23/23 - 0s - 7ms/step - loss: 1538.5912 - val_loss: 1563.8210
Epoch 3/100
23/23 - 0s - 7ms/step - loss: 1523.6218 - val_loss: 1548.9133
Epoch 4/100
23/23 - 0s - 8ms/step - loss: 1508.9130 - val_loss: 1533.5885
Epoch 5/100
23/23 - 0s - 7ms/step - loss: 1493.9691 - val_loss: 1518.1794
Epoch 6/100
23/23 - 0s - 6ms/step - loss: 1478.6296 - val_loss: 1502.5957
Epoch 7/100
23/23 - 0s - 6ms/step - loss: 1462.9534 - val_loss: 1486.3555
Epoch 8/100
23/23 - 0s - 6ms/step - loss: 1446.5956 - val_loss: 1469.4159
Epoch 9/100
23/23 - 0s - 6ms/step - loss: 1429.5973 - val_loss: 1451.4684
Epoch 10/100
23/23 - 0s - 5ms/step - loss: 1411.3938 - val_loss: 1432.9194
Epoch 11/100
23/23 - 0s - 7ms/step - loss: 1392.2120 - val_loss: 1413.3970
Epoch 12/100
23/23 - 0s - 6ms/step - loss: 1372.1455 - val_loss: 1392.4176
Epoch 13/100
23/23 - 0s - 7ms/step - loss: 1350.6511 - val_loss: 1370.3921
Epoch 14/100
23/23 - 0s - 6ms/step - loss: 13

In [29]:
# Compute the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mean_squared_errors)
std_mse = np.std(mean_squared_errors)

print("Mean of Mean Squared Errors: {}".format(mean_mse))
print("Standard Deviation of Mean Squared Errors: {}".format(std_mse))

Mean of Mean Squared Errors: 185.1879049682617
Standard Deviation of Mean Squared Errors: 13.496105397493086


# D. Increase the number of hidden layers

Repeat part B (set epoch=50) but with updated regression model

In [30]:
predictors = data.drop('Strength',axis=1)
target = data['Strength']

predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [31]:
# define regression model with 3 hidden layers with 10 nodes each
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))

    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [32]:
n_cols = predictors_norm.shape[1] # number of predictors
mean_squared_errors = []

for i in range(50):
    x_train, x_test, y_train, y_test = train_test_split(predictors_norm, target, test_size = 0.3, random_state = 1)
    model = regression_model()
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, verbose=2)
    mse = model.evaluate(x_test, y_test, verbose=0)
    mean_squared_errors.append(mse)

Epoch 1/50


/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 48ms/step - loss: 1538.7347 - val_loss: 1568.6195
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1518.4723 - val_loss: 1546.7863
Epoch 3/50
23/23 - 0s - 8ms/step - loss: 1497.2004 - val_loss: 1522.8291
Epoch 4/50
23/23 - 0s - 8ms/step - loss: 1473.0515 - val_loss: 1495.4111
Epoch 5/50
23/23 - 0s - 7ms/step - loss: 1444.8435 - val_loss: 1462.0162
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1410.0372 - val_loss: 1421.7296
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1367.0593 - val_loss: 1371.0575
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1312.6436 - val_loss: 1306.3888
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1243.6503 - val_loss: 1225.6509
Epoch 10/50
23/23 - 0s - 7ms/step - loss: 1157.7257 - val_loss: 1127.0435
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1054.9747 - val_loss: 1008.6287
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 931.9142 - val_loss: 868.0237
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 791.1966 - val_loss: 718.0402
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 649.4167 - val_loss

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 48ms/step - loss: 1530.3610 - val_loss: 1548.4281
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1501.9204 - val_loss: 1517.5823
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1471.4600 - val_loss: 1484.0924
Epoch 4/50
23/23 - 0s - 8ms/step - loss: 1437.4053 - val_loss: 1444.4432
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1397.1180 - val_loss: 1397.1074
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1347.3450 - val_loss: 1341.0234
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1288.8695 - val_loss: 1274.0002
Epoch 8/50
23/23 - 0s - 7ms/step - loss: 1219.8977 - val_loss: 1196.2655
Epoch 9/50
23/23 - 0s - 7ms/step - loss: 1140.2325 - val_loss: 1110.1666
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1052.6483 - val_loss: 1015.2051
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 957.0885 - val_loss: 916.2808
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 858.6547 - val_loss: 816.8424
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 761.1920 - val_loss: 719.6510
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 667.4440 - val_loss: 

...

23/23 - 1s - 45ms/step - loss: 1586.9696 - val_loss: 1611.3787
Epoch 2/50
23/23 - 0s - 6ms/step - loss: 1565.8630 - val_loss: 1592.7045
Epoch 3/50
23/23 - 0s - 6ms/step - loss: 1548.2682 - val_loss: 1575.0970
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1530.8076 - val_loss: 1556.7401
Epoch 5/50
23/23 - 0s - 7ms/step - loss: 1511.9792 - val_loss: 1535.5791
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1490.3027 - val_loss: 1510.1719
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1463.8569 - val_loss: 1478.3788
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1430.3027 - val_loss: 1438.4005
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 1387.6333 - val_loss: 1387.7705
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1334.2451 - val_loss: 1322.4160
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1265.5538 - val_loss: 1242.0439
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 1178.9794 - val_loss: 1141.3308
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 1074.0791 - val_loss: 1023.2597
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 956.0750 - val_

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 45ms/step - loss: 1527.8429 - val_loss: 1554.6313
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1511.6097 - val_loss: 1536.8602
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1493.8213 - val_loss: 1516.3325
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1472.1874 - val_loss: 1490.1150
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1444.0544 - val_loss: 1456.0729
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1408.0228 - val_loss: 1412.6876
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1361.2794 - val_loss: 1359.0510
Epoch 8/50
23/23 - 0s - 7ms/step - loss: 1303.9440 - val_loss: 1292.4437
Epoch 9/50
23/23 - 0s - 7ms/step - loss: 1233.0554 - val_loss: 1212.5996
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 1149.0708 - val_loss: 1116.6481
Epoch 11/50
23/23 - 0s - 6ms/step - loss: 1048.5743 - val_loss: 1010.2842
Epoch 12/50
23/23 - 0s - 5ms/step - loss: 938.3666 - val_loss: 898.7426
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 825.9385 - val_loss: 783.5920
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 711.7828 - val_loss

/opt/conda/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 - 1s - 43ms/step - loss: 1506.1292 - val_loss: 1530.6619
Epoch 2/50
23/23 - 0s - 7ms/step - loss: 1477.4062 - val_loss: 1499.0120
Epoch 3/50
23/23 - 0s - 7ms/step - loss: 1441.7803 - val_loss: 1458.5436
Epoch 4/50
23/23 - 0s - 7ms/step - loss: 1396.4110 - val_loss: 1406.2999
Epoch 5/50
23/23 - 0s - 6ms/step - loss: 1337.9725 - val_loss: 1339.2896
Epoch 6/50
23/23 - 0s - 6ms/step - loss: 1264.7661 - val_loss: 1256.2334
Epoch 7/50
23/23 - 0s - 6ms/step - loss: 1177.2653 - val_loss: 1159.5304
Epoch 8/50
23/23 - 0s - 6ms/step - loss: 1078.0851 - val_loss: 1050.0104
Epoch 9/50
23/23 - 0s - 6ms/step - loss: 966.8294 - val_loss: 933.0526
Epoch 10/50
23/23 - 0s - 6ms/step - loss: 850.2321 - val_loss: 807.7118
Epoch 11/50
23/23 - 0s - 5ms/step - loss: 728.1658 - val_loss: 681.4622
Epoch 12/50
23/23 - 0s - 6ms/step - loss: 605.9975 - val_loss: 563.2062
Epoch 13/50
23/23 - 0s - 6ms/step - loss: 493.1258 - val_loss: 455.7512
Epoch 14/50
23/23 - 0s - 6ms/step - loss: 396.1082 - val_loss: 366.

In [35]:
# Compute the mean and standard deviation of the mean squared errors
mean_mse = np.mean(mean_squared_errors)
std_mse = np.std(mean_squared_errors)

print("Mean of Mean Squared Errors: {}".format(mean_mse))
print("Standard Deviation of Mean Squared Errors: {}".format(std_mse))

Mean of Mean Squared Errors: 168.9237518310547
Standard Deviation of Mean Squared Errors: 6.044861437483092


In [ ]:
##############

### Results
| Experiment | Epochs | Hidden Layers | Normalization | Mean MSE | Std. Dev. MSE |
|------------|--------|--------------|--------------|----------|--------------|
| Baseline Model | 50 | 1 | No | 366.7523 | 555.3843 |
| Normalized Data | 50 | 1 | Yes | 356.3774 | 66.6283 |
| Increased Epochs | 100 | 1 | Yes | 185.1879 | 13.4961 |
| Increased Hidden Layers | 50 | 3 | Yes | 168.9238 | 6.0449 |

### **Analysis Based on Results Table**  

#### **1. Effect of Normalization**  
- Normalizing the data slightly reduced the Mean Squared Error (MSE) from **366.75** to **356.38**.  
- More importantly, the standard deviation of MSE significantly decreased from **555.38** to **66.63**, indicating more stable and consistent predictions.  
- This suggests that normalization improves model performance by ensuring better numerical stability and faster convergence.  

#### **2. Effect of Increasing Epochs**  
- Increasing the number of training epochs from **50 to 100** dramatically reduced the MSE from **356.38** to **185.19**.  
- The standard deviation also dropped from **66.63** to **13.50**, indicating improved model consistency.  
- This result shows that training for more epochs allows the model to learn better, reducing error significantly.  

#### **3. Effect of Increasing Hidden Layers**  
- Adding more hidden layers (3 instead of 1) further reduced the MSE from **185.19** to **168.92**.  
- The standard deviation also improved from **13.50** to **6.04**, showing increased stability.  
- This suggests that a deeper network captures more complex patterns, improving accuracy while keeping variability low.  

### **Conclusion**  
- **Normalization** helps with stability and slight MSE reduction.  
- **Increasing epochs** drastically lowers MSE, showing the model benefits from longer training.  
- **Adding more hidden layers** further improves performance, indicating that a deeper model better captures the complexity of the data.  
- The **best configuration** in this case appears to be **3 hidden layers, 50 epochs, with normalization**, as it achieves the lowest MSE and highest stability.  
